<a href="https://colab.research.google.com/github/noumantechie/langchain/blob/main/memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**long-Term memory to LLM using redis **

In [ ]:
%pip install -qU langchain-google-genai

In [ ]:
%pip install -qU langchain-redis langchain_google_genai redis

**Modules that need for storing chat history **

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI


from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_redis import RedisChatMessageHistory
# from langchain_core.runnables import ConfigurableField

In [ ]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=GOOGLE_API_KEY  # Corrected 'api_ket' to 'api_key'
)


In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an assistant who's good at {ability}. Respond in 20 words or fewer."),
        MessagesPlaceholder(variable_name="history"),  # Moved inside the list
        ("human", "{input}"),
    ]
)

base_chain = prompt | llm



**Redis URL where it is running and here is list where we store history**

In [ ]:
REDIS_URL = "redis://localhost:6379/0"
store = {}

**Every session is going to have a session id so, the idea is we can have multiple threads and each thread will have a different thread id or session id . So, we need to retrueve that session id if it does not exist then we need to create that.**

In [ ]:
def get_message_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(
        session_id=session_id,
        redis_url=REDIS_URL  # Correct parameter name
    )

redis_chain = RunnableWithMessageHistory(
    base_chain,
    get_message_history,
    input_messages_key="input",
    history_messages_key="history"
)



In [ ]:
redis_chain.invoke(
    {"ability": "math",
     "input": "what does cosine mean?"},
    config={"configurable": {"session_id": "math_thread1"}},
)


In [ ]:
redis_chain.invoke(
    {"ability": "math",
     "input": "Tell me more?"},
    config={"configurable": {"session_id": "math_thread1"}},
)

**For Multiple Thread Running at same time **

In [ ]:
redis_chain.invoke(
    {"ability": "physics",
     "input": "what is the therory of relativity?"},
    config={"configurable": {"session_id": "phy_thread1"}},
)

In [ ]:
redis_chain.invoke(
    {"ability": "physics",
     "input": "Tell me more"},
    config={"configurable": {"session_id": "phy_thread1"}},
)